In [ ]:
import requests
import csv
from google.colab import drive
drive.mount('/content/drive')

In [4]:
headers = {
    'x-rapidapi-key': "7f7a73e0c7msh3a250c73c1e427cp179969jsn791ff966f28f",
    'x-rapidapi-host': "edamam-food-and-grocery-database.p.rapidapi.com"
    }

r = requests.get("https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser", headers=headers)
r.status_code

200

In [1]:
def extract_champagne_products():
    url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

    querystring = {"ingr":"champagne"}

    headers = {
        'x-rapidapi-key': "7f7a73e0c7msh3a250c73c1e427cp179969jsn791ff966f28f",
        'x-rapidapi-host': "edamam-food-and-grocery-database.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()

    products = []
    verb = [] # log des erreurs
    for i in range(min(10, len(data['hints']))):
        product_info = data['hints'][i]['food']
        food_id = product_info['foodId']
        label = product_info['label']
        category = product_info['category']
        try:
          food_contents_label = product_info['foodContentsLabel']
        except KeyError:
          food_contents_label = None
          verb.append(f'No foodContentsLabel in product {food_id}')
        try:
          image = product_info['image']
        except KeyError:
          image = None
          verb.append(f'No image in product {food_id}')

        products.append({
            'foodId': food_id,
            'label': label,
            'category': category,
            'foodContentsLabel': food_contents_label,
            'image': image
        })

    return products

path_data = '/content/drive/MyDrive/FORMATION DATASCIENTIST OC - Lucas GAMBA/Projet 6/'

def save_to_csv(products, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['foodId', 'label', 'category', 'foodContentsLabel', 'image']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for product in products:
            writer.writerow(product)

if __name__ == "__main__":
    champagne_products = extract_champagne_products()
    save_to_csv(champagne_products, path_data + 'champagne_products.csv')


Mounted at /content/drive


In [ ]:
# test affichage sous pandas
import pandas as pd

champagne_table = pd.read_csv(path_data + 'champagne_products.csv')
champagne_table

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,NaN
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN


Autre approche plus compact utilisant la méthode json_normalize de pandas :

In [28]:
import pandas as pd

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

querystring = {"ingr":"champagne"}

headers = {
    'x-rapidapi-key': "7f7a73e0c7msh3a250c73c1e427cp179969jsn791ff966f28f",
    'x-rapidapi-host': "edamam-food-and-grocery-database.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()

print(len(data['hints']), ' produits trouvés.')

tab_champ = pd.json_normalize(data['hints'])[['food.foodId', 'food.label', 'food.category', 'food.foodContentsLabel', 'food.image']]
print("Voici un dataframe listant les dix premiers :")
print("")
tab_champ.head(10)

20  produits trouvés.
Voici un dataframe listant les dix premiers :



,food.foodId,food.label,food.category,food.foodContentsLabel,food.image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,NaN
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
